In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import pandas as pd

train_df = pd.read_parquet('/content/drive/MyDrive/Alzheimer Data/Train.parquet')
test_df = pd.read_parquet('/content/drive/MyDrive/Alzheimer Data/Test.parquet')

print( train_df.columns )
print( test_df.columns )

Index(['image', 'label'], dtype='object')
Index(['image', 'label'], dtype='object')


In [9]:
print( train_df['image'].iloc[0] ) # Since we figured each image is a dictionary we will print it out

{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x01,\x01,\x00\x00\xff\xdb\x00C\x00\x03\x02\x02\x03\x02\x02\x03\x03\x03\x03\x04\x03\x03\x04\x05\x08\x05\x05\x04\x04\x05\n\x07\x07\x06\x08\x0c\n\x0c\x0c\x0b\n\x0b\x0b\r\x0e\x12\x10\r\x0e\x11\x0e\x0b\x0b\x10\x16\x10\x11\x13\x14\x15\x15\x15\x0c\x0f\x17\x18\x16\x14\x18\x12\x14\x15\x14\xff\xc0\x00\x0b\x08\x00\x80\x00\x80\x01\x01\x11\x00\xff\xc4\x00\x1d\x00\x00\x02\x03\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x07\x05\x06\x08\x04\x02\x03\x01\t\xff\xc4\x008\x10\x00\x01\x03\x03\x03\x02\x05\x03\x01\x05\x08\x03\x00\x00\x00\x00\x01\x02\x03\x04\x00\x05\x11\x06\x07\x12!1\x13"AQa\x142q\x81\x08\x15R\x91\xa1#$3Bb\x92\xb1\xd1\x16r\x82\xff\xda\x00\x08\x01\x01\x00\x00?\x00\xfeUQE\x14QE\x14QE\x14QE\x14QE\x14W\xea\x1bS\x8bJ\x12\x92\xa5(\xe0$\x0c\x92\x7f\x157\x07BjK\x9a\x1c\\;\x05\xceRZ\xfb\xd4\xd4G\x14\x13\xf9\xc0\xe9\xd8\xd4l\xfbL\xdbZ\xf8\xcc\x86\xfcU{<\xd9G\xcf\xadu\xda\xb4\x95\xee\xfaH\xb6\xd9\xe7O\xc63\xf4\xd1\x96\xe63\xdb\xb0\xae\x

In [10]:
import numpy as np
import cv2
from PIL import Image
from io import BytesIO

IMG_SIZE = 128 # Resized images-scale
TOTAL_CLASSES = 4 # Total classes in this model

def process_image(bin_image):
    # Decode a binary image into a matrice
    img = Image.open(BytesIO(bin_image['bytes']))
    img = img.resize((IMG_SIZE, IMG_SIZE)) # We neglect aspect ratio differs
    img = img.convert('RGB') # Ensure 3 color channels
    img_arr = np.array(img)
    return img_arr

# Process all dataframe images
train_images = np.array([ process_image(img_data) for img_data in train_df['image'] ])
test_images = np.array([ process_image(img_data) for img_data in test_df['image'] ])

# Normalize all pixel colors
train_images = train_images / 255.0
test_images = test_images / 255.0

# Extract all image labels
train_labels = train_df['label'].values
test_labels = test_df['label'].values

In [11]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Since we are working on a multi-class classification we must use categorical regression
le = LabelEncoder()
train_labels = le.fit_transform(train_labels)
test_labels = le.transform(test_labels)

# All labels are now converted into one-hot encoding for all different classes
train_labels = to_categorical(train_labels, num_classes=TOTAL_CLASSES)
test_labels = to_categorical(test_labels, num_classes=TOTAL_CLASSES)

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Now we will finally construct the convolutional layers of our neural network
# The flow of our network is going from small convulutional filters into bigger ones

# We prefer relu over other activation functions due to optimization purposes.
model = Sequential([
    # 1st convolution layer
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D((2, 2)),

    # 2nd convolution layer
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    # 3rd convolutional layer
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    # Flatten the output and convert all features into vectors
    Flatten(),

    # Fully connected layer
    Dense(128, activation='relu'),
    Dropout(0.5), # Avoid overfitting by making the model not overly sensitive to noise

    # Output layer
    Dense(TOTAL_CLASSES, activation='softmax') # Finally output the classifer out of all 4 classes
])

model.summary() # Visualize the model's contents

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,305,156 (12.61 MB)

 Trainable params: 3,305,156 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
model.compile(
    optimizer='adam', # Optimizer
    loss='categorical_crossentropy',
    metrics=['accuracy'] # Monitor solely the accuracy of the model
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Prevent overfitting during the training part when the model does not get substantially more accurate
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# We finally can train our CNN model
history = model.fit(
    train_images, # All the matrice images
    train_labels, # All the labels
    epochs=20, # The amount of repeats for the backpropogation process
    batch_size=32,
    validation_split=0.2, # Give in 20% for validation
    shuffle=True, # Randomly iterate through the images to not provide a heuristic bias
    callbacks=[early_stopping] # Apply an early stopping callback to prevent overfitting
)

model.save('/content/drive/My Drive/Trained Models/Alzheimer')

Epoch 1/20
123/128 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.5774 - loss: 0.9043

In [17]:
# After our model is finally trained, we can now predict the model on unique images it has likely never seen
def preprocess_image(image_path):
    # Preprocesses a prediction image
    img = Image.open(image_path)
    img = img.resize((IMG_SIZE, IMG_SIZE))  # Resize to match model input size
    img = img.convert('RGB')  # Ensure the image has 3 channels
    img_arr = np.array(img) / 255.0  # Normalize the image color channels
    img_arr = np.expand_dims(img_arr, axis=0)  # Add batch dimension
    return img_arr

In [24]:
test_image = preprocess_image('/content/normal_brain.jpg') # Ensure there is a valid image path
predictions = model.predict(test_image)
print(predictions) # We can finally see the predictions of our model!

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
[[0.03860325 0.00441601 0.6017757  0.3552051 ]]
[0. 0. 0. 1.]
